# Let's Fetch **The Price of Fuel in India Today**

In [4]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from pathlib import Path
from datetime import datetime
import time

fuels = ['petrol', 'diesel', 'lpg', 'cng']
source_page = lambda fuel: f"https://www.goodreturns.in/{fuel}-price.html"

In [5]:
# Selenium options required to create a 'headless' browser
options = Options()
options.add_argument("--blink-settings=imagesEnabled=false")
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.37")

driver = webdriver.Chrome(options=options)

In [6]:
def city_price(result):
    city = result.split('₹')
    price = city[1].split(' ')[0]
    city = city[0].strip()
    return city, price

def get_fuel_prices(fuel):
    fuel = fuel_prices[fuel.lower()]
    df = pd.DataFrame(fuel.items(), columns=["City", "Price"])
    df['Price'] = df['Price'].astype(float)
    df = df.sort_values(by='Price', ascending=False)
    df = df.reset_index(drop=True)
    return df

def display_fuel_prices(fuel):
    print(f"{fuel.upper()} PRICES AS ON {datetime.now().strftime('%d %B %Y')}")
    print(f"Source: {source_page(fuel)}")
    print("-" * 35)
    display(get_fuel_prices(fuel))

In [7]:
fuel_prices = {}

for fuel in fuels:
    driver.get(source_page(fuel))
    # time.sleep(5)  # Allow extra time for JavaScript to load
    results = driver.find_element(By.CLASS_NAME, "gd-fuel-table-block")
    results = results.text.split("\n")[2:]

    fuel_prices[fuel] = {}
    for result in results:
        city, price = city_price(result)
        fuel_prices[fuel][city] = price

In [11]:
display_fuel_prices('petrol')

PETROL PRICES AS ON 26 July 2025
Source: https://www.goodreturns.in/petrol-price.html
-----------------------------------


,City,Price
0,Thiruvananthapuram,107.48
1,Hyderabad,107.46
2,Kolkata,105.41
3,Patna,105.23
4,Jaipur,104.69
5,Mumbai,103.50
6,Bangalore,102.92
7,Bhubaneswar,101.11
8,Chennai,100.80
9,Gurgaon,95.41


In [10]:
display_fuel_prices('lpg')

LPG PRICES AS ON 26 July 2025
Source: https://www.goodreturns.in/lpg-price.html
-----------------------------------


,City,Price
0,Patna,942.5
1,Hyderabad,905.0
2,Lucknow,890.5
3,Kolkata,879.0
4,Bhubaneswar,879.0
5,Chennai,868.5
6,Chandigarh,862.5
7,Thiruvananthapuram,862.0
8,Gurgaon,861.5
9,Jaipur,856.5


In [12]:
display_fuel_prices('diesel')

DIESEL PRICES AS ON 26 July 2025
Source: https://www.goodreturns.in/diesel-price.html
-----------------------------------


,City,Price
0,Thiruvananthapuram,96.48
1,Hyderabad,95.70
2,Bhubaneswar,92.69
3,Chennai,92.39
4,Kolkata,92.02
5,Patna,91.49
6,Bangalore,90.99
7,Jaipur,90.18
8,Mumbai,90.03
9,Noida,87.89
